## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Mataura,NZ,-46.1927,168.8643,73.92,36,100,4.09,overcast clouds
1,1,Peniche,PT,39.3558,-9.3811,61.61,94,96,6.80,overcast clouds
2,2,Nikolskoye,RU,59.7035,30.7861,28.02,82,11,6.62,few clouds
3,3,Ugoofaaru,MV,5.6667,73.0000,80.60,81,99,11.48,overcast clouds
4,4,Sao Joao Da Barra,BR,-21.6403,-41.0511,71.76,80,100,4.21,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))

What is your desired minimum temperature for your trip? 70
What is your desired maximum temperature for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) &
                                       (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Mataura,NZ,-46.1927,168.8643,73.92,36,100,4.09,overcast clouds
3,3,Ugoofaaru,MV,5.6667,73.0000,80.60,81,99,11.48,overcast clouds
4,4,Sao Joao Da Barra,BR,-21.6403,-41.0511,71.76,80,100,4.21,overcast clouds
8,8,Atuona,PF,-9.8000,-139.0333,77.77,79,15,16.91,light rain
13,13,Vaini,TO,-21.2000,-175.2000,82.56,89,75,9.22,moderate rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                249
City                   249
Country                249
Lat                    249
Lng                    249
Max Temp               249
Humidity               249
Cloudiness             249
Wind Speed             249
Weather Description    249
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna().copy()
clean_df.count()

City_ID                249
City                   249
Country                249
Lat                    249
Lng                    249
Max Temp               249
Humidity               249
Cloudiness             249
Wind Speed             249
Weather Description    249
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Mataura,NZ,73.92,overcast clouds,-46.1927,168.8643,
3,Ugoofaaru,MV,80.60,overcast clouds,5.6667,73.0000,
4,Sao Joao Da Barra,BR,71.76,overcast clouds,-21.6403,-41.0511,
8,Atuona,PF,77.77,light rain,-9.8000,-139.0333,
13,Vaini,TO,82.56,moderate rain,-21.2000,-175.2000,
16,Nioro,GM,75.09,scattered clouds,13.3500,-15.7500,
20,San Patricio,US,73.83,clear sky,28.0170,-97.5169,
24,Razole,IN,77.29,overcast clouds,16.4833,81.8333,
30,Rikitea,PF,74.80,light rain,-23.1203,-134.9692,
32,Bambous Virieux,MU,75.45,light rain,-20.3428,57.7575,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
print("----- Finished -----")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
----- Finished -----


In [10]:
hotel_df.count()

City                   249
Country                249
Max Temp               249
Weather Description    249
Lat                    249
Lng                    249
Hotel Name             249
dtype: int64

In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)
clean_hotel_df = hotel_df.copy()
clean_hotel_df.count()

City                   249
Country                249
Max Temp               249
Weather Description    249
Lat                    249
Lng                    249
Hotel Name             249
dtype: int64

In [15]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Mataura,NZ,73.92,overcast clouds,-46.1927,168.8643,Ellie's Villa
3,Ugoofaaru,MV,80.60,overcast clouds,5.6667,73.0000,Ungoofaaru Inn
4,Sao Joao Da Barra,BR,71.76,overcast clouds,-21.6403,-41.0511,Pousada Porto De Canoas
8,Atuona,PF,77.77,light rain,-9.8000,-139.0333,Villa Enata
13,Vaini,TO,82.56,moderate rain,-21.2000,-175.2000,Keleti Beach Resort
16,Nioro,GM,75.09,scattered clouds,13.3500,-15.7500,
20,San Patricio,US,73.83,clear sky,28.0170,-97.5169,"Motel 6 Sinton, TX"
24,Razole,IN,77.29,overcast clouds,16.4833,81.8333,Sterling Palavelli Godavari
30,Rikitea,PF,74.80,light rain,-23.1203,-134.9692,People ThankYou
32,Bambous Virieux,MU,75.45,light rain,-20.3428,57.7575,Casa Tia Villa


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))